In [259]:
import numpy as np
import copy

class Minemap():
    def __init__(self, hei, wid, mine, density=3, is_safe_start=True):
        self.hei = hei
        self.wid = wid
        self.mine = mine
        self.density = density
        self.is_safe_start = is_safe_start
        self.nb_cancelled = 0
        self.travel = 0
        self.travel_prev_x = -1
        self.travel_prev_y = -1
        
        self.target_x = 0
        self.target_y = 0
        self.cell_map = []
        self.luck_map = []
        self.bomb_map = []

    def up_neighbour(self, x, y):
        for i in range( -(x != 0), 1 + (x != self.hei - 1)):
            for j in range( -(y != 0), 1 + (y != self.wid - 1)):
                self.cell_map[x+i][y+j] += 1
        
    def get_neighbour(self, x, y, value=-1):
        res = 0
        for i in range( -(x != 0), 1 + (x != self.hei - 1)):
            for j in range( -(y != 0), 1 + (y != self.wid - 1)):
                res += (self.cell_map[x + i][y + j] == value)
        return (res)

    def travel(self, cell_map, x, y, start_x, start_y):
        pass

    def is_placeable(self, x, y):
        return 1
        neigh_bomb += (x == 0) + (x == hei-1) + (y == 0) + (y == wid-1)
        if x > 0:
            neigh_bomb += (self.cell_map[x-1][y] == -1)
        if x < hei-1:
            neigh_bomb += (self.cell_map[x+1][y] == -1)
        if y > 0:
            neigh_bomb += (self.cell_map[x][y-1] == -1)
        if y < wid-1:
            neigh_bomb += (self.cell_map[x][y+1] == -1)
        if neigh_bomb <= 1:
            return 1
        self.cell_map[x][y] = -1 #be careful with this
        return 1

    def get_distance(self, x, y, pos_x, pos_y):
        return ((x-pos_x)**2+(y-pos_y)**2)**0.5

    def lower_lucky(self, x, y):
        max_val = 0
        #print(min_x, min_y, max_x, max_y)
        for i in range(0, self.hei):
            for j in range(0, self.wid):
                dist = self.get_distance(x, y, i, j)
                incr = dist**(1/self.density)
                self.luck_map[i][j] = incr
                if max_val < incr:
                    max_val = incr
        return max_val

    def generate_map_v1(self):
        empty_cell = self.hei * self.wid
        self.cell_map = np.zeros((self.hei, self.wid))
        self.luck_map = np.zeros((self.hei, self.wid))
        self.bomb_map = np.zeros((self.hei, self.wid))
        nb = [i for i in range(empty_cell)]
        stop = len(nb)
        self.nb_cancelled = 0
        max_val = 1
        checkme = []
        
        while empty_cell+self.mine > stop:
            pos = np.random.randint(0, empty_cell)
            rdm = nb[pos] #random between 0 and amount of empty cell to get an empty cell value
            x = rdm // self.wid #get column
            y = rdm % self.wid #get line
            if np.random.random()*max_val > self.luck_map[x][y]:# and is_placeable(copy.copy(cell_map), x, y, hei, wid) == 1:
                self.bomb_map[x][y] = 1 #put mine at coordinates
                self.up_neighbour(x, y)
                #print(rdm)
                #print(self.luck_map, "\n")
                empty_cell -= 1
                nb[pos] = nb[empty_cell] #swap the targeted cell (which is now a mine) with the last element of the array
                max_val = self.lower_lucky(x, y)
                if self.travel_prev_x != -1:
                    self.travel += self.get_distance(x, y, self.travel_prev_x, self.travel_prev_y)
                self.travel_prev_x = x
                self.travel_prev_y = y
            else:
                self.nb_cancelled += 1

        for x in range(self.hei):
            for y in range(self.wid):
                if self.bomb_map[x][y] != 0:
                    self.cell_map[x][y] = -1
        for idx, i in enumerate(checkme[0:-2]):
            if i in checkme[idx+1:]:
                print(i)
        #print(self.luck_map)
        
    def print_map(self):
        for i in self.cell_map:
            line = ""
            for j in i:
                if j == -1:
                    line += "X"
                if j >= 0:
                    line += " "
            line += "|"
            for j in i:
                if j == -1:
                    line += "X"
                if j == 0:
                    line += " "
                if j > 0:
                    line += str(int(j))
            line += "|"
            print(line)
        print("-"*len(self.cell_map)+"+"+"-"*len(self.cell_map))
        return self.travel
    
    def check(self):
        count = 0
        for i in self.cell_map:
            for j in i:
                count += (j == -1)
        if count != self.mine:
            self.print_map()
            return count
        return 0
    
    def map_density(self):
        res = 0
        for i in range(self.hei):
            for j in range(self.wid):
                res += self.get_neighbour(i,j)**2
        return res

In [261]:
limit = 100
bomb = 50
for density in [1, 2, 5, 10, 100]:
    avg = 0
    for i in range(limit):
        demineur = Minemap(20, 20, bomb, density)
        demineur.generate_map_v1()
        #demineur.print_map()
        avg += demineur.travel
    print(avg/limit/bomb, demineur.map_density(), bomb, density)

8.095188758978953 1016 50 1
7.602076195927689 899 50 2
7.305371702277096 876 50 5
7.1058957833523415 848 50 10
7.001938167521708 903 50 100


In [ ]:
demineur = Minemap(20, 20, 50, 1)
demineur.generate_map_v1()
demineur.print_map()
print(demineur.map_density(), demineur.travel)

demineur = Minemap(20, 20, 50, 100)
demineur.generate_map_v1()
demineur.print_map()
print(demineur.map_density(), demineur.travel)